# libraries

In [1]:
import requests
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

## OPTIONS DATA

In [13]:
def updateData():
    ''' This function is used to get the raw-data in JSON format from NSE NIFTY50 Options using it's API and storing it '''
    
    url='https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY'
    headers={
        'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
        "accept-encoding" : "gzip, deflate, br",
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8'
    }
    session=requests.Session()
    request=session.get(url,headers=headers)
    cookies=dict(request.cookies)
    response=session.get(url,headers=headers,cookies=cookies).json()
    rawdata=pd.DataFrame(response)
    # The rawdata would have the present day's record as well as future dateabss
    rawop=pd.DataFrame(rawdata['filtered']['data']).fillna(0)
    nifty50cur=rawdata['records']['underlyingValue']
    # Taking only the present day's data, and returning it
    return rawop,nifty50cur

## option data extraction

In [14]:
def dataframe(rawop):
    ''' This function is used to convert the Raw-JSON data into a structured data-frame'''
    data=[]
    for i in range(0,len(rawop)):
        calloi=callcoi=putoi=putcoi=calliv=putiv=cltp=pltp=0
        stp=rawop['strikePrice'][i]
        exp=rawop["expiryDate"][i]
        if(rawop['CE'][i]==0):
            calloi=callcoi=0
        else:
            calloi=rawop['CE'][i]['openInterest']
            callcoi=rawop['CE'][i]['changeinOpenInterest']            
            cltp=rawop['CE'][i]['lastPrice']
            calliv=rawop['CE'][i]['impliedVolatility']
        if(rawop['PE'][i]==0):
            putoi=putcoi=0
        else:
            putoi=rawop['PE'][i]['openInterest']
            putcoi=rawop['PE'][i]['changeinOpenInterest']            
            pltp=rawop['PE'][i]['lastPrice']
            putiv=rawop['PE'][i]['impliedVolatility']
        opdata = {
            'EXP': exp,
            'STP': stp,
            'CALL OI': calloi, 'CALL COI':callcoi, 'CALL LTP':cltp, 'CALL IV':calliv,
            'PUT OI': putoi, 'PUT COI':putcoi, 'PUT LTP':pltp, 'PUT IV':putiv,
        }              
        data.append(opdata)
    optionchain=pd.DataFrame(data)
    return optionchain

In [5]:
def main():
    old=pd.read_csv(oldData.csv)
    rawop=updateData()
    niftyprice=updateNifty()
    optionchain=dataframe(rawop)
    optionchain=appendpcrOI(optionchain)
    optionchain=appendpcrCOI(optionchain)
    STPrange=STP_range(optionchain, 5, niftyprice)
    plotOI(optionchain,per,STPrange)
    plotCOI(optionchain,per,STPrange)
    plotIV(optionchain,per,STPrange)
    old.loc['CalloldIVATM','PutoldIVATM']=storeIV(optionchain,CalloldIVATM,PutoldIVATM)
    plotIVtime(optionchain)
    print(totPCR(optionchain))

In [15]:
dataframe(updateData())

,EXP,STP,CALL OI,CALL COI,CALL LTP,CALL IV,PUT OI,PUT COI,PUT LTP,PUT IV
0,15-Sep-2022,14050,0,0,0.00,0.00,1220,571,0.60,63.62
1,15-Sep-2022,14100,0,0,0.00,0.00,639,100,0.60,62.75
2,15-Sep-2022,14150,0,0,0.00,0.00,58,12,0.70,62.75
3,15-Sep-2022,14200,0,0,0.00,0.00,498,416,0.65,61.46
4,15-Sep-2022,14250,0,0,0.00,0.00,47,35,0.70,61.00
...,...,...,...,...,...,...,...,...,...,...
104,15-Sep-2022,19250,542,244,0.90,22.79,0,0,0.00,0.00
105,15-Sep-2022,19300,9259,5706,0.80,23.19,0,0,0.00,0.00
106,15-Sep-2022,19350,1255,907,0.80,23.88,0,0,0.00,0.00
107,15-Sep-2022,19400,20157,7409,0.80,24.56,0,0,0.00,0.00


In [6]:
while True:
    main()
    time.sleep(600)

NameError: name 'oldData' is not defined

In [17]:
def appendpcrOI(optionchain):
    ''' This function gives us the ratio for PUT OI and CALL OI.
        Significance is to get what is the people's perception of a particular strike price'''
    pcr=[]
    optionchain=optionchain.T
    for i in range(0,optionchain.shape[1]):
        if(optionchain.loc['CALL OI'][i]!=0):
            pcr.append(optionchain.loc['PUT OI'][i]/optionchain.loc['CALL OI'][i])
        else:
            pcr.append(0)
    optionchain.loc['PCR OI']=pcr
    optionchain=optionchain.T
    return optionchain

In [19]:
def appendpcrCOI(optionchain):
    '''This function provides us the ratio of the change in PUT OI and the change in the CALL OI.
       This function tells us the dynamic change in the people's percetion'''
    pcr=[]
    optionchain=optionchain.T
    for i in range(0,optionchain.shape[1]):
        if(optionchain.loc['CALL COI'][i]!=0):
            pcr.append(optionchain.loc['PUT COI'][i]/optionchain.loc['CALL COI'][i])
        else:
            pcr.append(0)
    optionchain.loc['PCR COI']=pcr
    optionchain=optionchain.T
    return optionchain

In [ ]:
def STP_range(optionchain, percent, niftyprice):
    optionchain=optionchain.T
    for i in range(0,optionchain.shape[1]-1):
        if((optionchain.loc['STP'][i]<((100-percent)/100)*niftyprice) & (optionchain.loc['STP'][i+1]>((100-percent)/100)*niftyprice)):
            min=i
        if((optionchain.loc['STP'][i]<((100+percent)/100)*niftyprice) & (optionchain.loc['STP'][i+1]>((100+percent)/100)*niftyprice)):
            max=i
    range1=[min,max]
    return (range1)

In [ ]:
def plotOI(optionchain,per,STPrange):
    [min,max]=STPrange
    X = optionchain['STP'][min:max]
    YC = optionchain['CALL OI'][min:max]
    YP = optionchain['PUT OI'][min:max]

    X_axis = np.arange(len(X))

    plt.bar(X_axis - 0.2, YC, 0.4, label = 'CALL')
    plt.bar(X_axis + 0.2, YP, 0.4, label = 'PUT')

    plt.xticks(X_axis, X)
    plt.xlabel("STP")
    plt.ylabel("OI")
    plt.title("CALL | PUT OI BAR GRAPH")
    plt.legend()
    plt.show()

    

In [ ]:
def plotCOI(optionchain,per,STPrange):
    [min,max]=STPrange
    X = optionchain['STP'][min:max]
    YC = optionchain['CALL COI'][min:max]
    YP = optionchain['PUT COI'][min:max]

    X_axis = np.arange(len(X))

    plt.bar(X_axis - 0.2, YC, 0.4, label = 'CALL')
    plt.bar(X_axis + 0.2, YP, 0.4, label = 'PUT')

    plt.xticks(X_axis, X)
    plt.xlabel("STP")
    plt.ylabel("COI")
    plt.title("CALL | PUT COI BAR GRAPH")
    plt.legend()
    plt.show()

In [ ]:
def totPCR(optionchain):
    totCall=sum(np.array(optionchain['CALL OI']))
    totPut=sum(np.array(optionchain['PUT OI']))
    return [(totPut/totCall),totPut,totCall]

In [ ]:
def plotIV(optionchain,per,STPrange):
    [min,max]=STPrange
    X = optionchain['STP'][min:max]
    YC = optionchain['CALL IV'][min:max]
    YP = optionchain['PUT IV'][min:max]

    X_axis = np.arange(len(X))

    plt.bar(X_axis - 0.2, YC, 0.4, label = 'CALL')
    plt.bar(X_axis + 0.2, YP, 0.4, label = 'PUT')

    plt.xticks(X_axis, X)
    plt.xlabel("STP")
    plt.ylabel("IV")
    plt.title("CALL | PUT IV BAR GRAPH")
    plt.legend()
    plt.show()
    

In [ ]:
def plotIVtime(optionchain):
  
    x = np.arange(0,len(CalloldIVATM)) #it is time
    y = CalloldIVATM #CalloldIVATM is a global variable
    # first plot with X and Y data
    plt.plot(x, y)
    y1 = PutoldIVATM

    # second plot with x1 and y1 data
    plt.plot(x, y1, '-.')

    plt.xlabel("time")
    plt.ylabel("IV ATM")
    plt.title('IV changing with time')
    plt.show()
    

In [ ]:
def storeIV(optionchain,CalloldIVATM,PutoldIVATM):
    optionchain=optionchain.T
    i=STPATM(optionchain)
    CalloldIVATM.append(optionchain.loc['CALL IV'][i])
    PutoldIVATM.append(optionchain.loc['PUT IV'][i])
    return ([CalloldIVATM,PutoldIVATM])  #add update iv atm in main function
    
    

In [ ]:
def STPATM(optionchain):
    for i in range(0,optionchain.shape[0]):
        if(optionchain.iloc[i][1]<currentNifty() && optionchain.iloc[i+1][1]>currentNifty()):
            return i